# Implementação de código baseado no artigo de Hannah Levin, Samir Agarwala e Juan Triana: Understanding Complex Emotions in Sentences

### Atividade referente à disciplina Processamento de Linguagem Natural, com a Professora Nádia Félix

*Alunas: Karla Riccioppo e Lívia Oliveira.*


---


> Explicações detalhadas de cada célula

> **1ª célula**

Esta célula é crucial para configurar o ambiente do projeto. Ela realiza as seguintes ações:
  1. Importação de Bibliotecas: Importa todas as bibliotecas Python que serão usadas nas etapas seguintes do projeto. Isso inclui:
      * torch, torch.nn, accelerate: Para deep learning e computação com GPUs.
      * transformers, BertTokenizer, BertModel: Essenciais para trabalhar com modelos pré-treinados como o BERT, que é a base do seu modelo de segmentação de emoções.
      * json, re, requests, numpy, tqdm, accuracy_score, collections.Counter: Para manipulação de dados, expressões regulares, requisições HTTP, operações numéricas, barras de progresso, avaliação de modelos e contagem de ocorrências, respectivamente.
  2. Inicialização do Dispositivo: Verifica se uma GPU (placa de vídeo) está disponível no ambiente (torch.cuda.is_available()). Se estiver, o modelo será executado na GPU para aproveitar o processamento paralelo e acelerar o treinamento. Caso contrário, ele usará a CPU. Isso é armazenado na variável device.
  3. Confirmação do Dispositivo: Imprime o dispositivo que será utilizado (🎯 Dispositivo: cpu no seu caso, pois a saída indica cpu) e, se for uma GPU, também informa o nome dela. Esta saída ajuda a confirmar que o ambiente está configurado corretamente.



In [ ]:
#1. importações e inicialização do modelo
!pip install transformers torch accelerate tqdm requests
import torch
import torch.nn as nn
import json
import re
import requests
import numpy as np
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from collections import Counter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎯 Dispositivo: {device}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name()}")

🎯 Dispositivo: cpu


> **2ª célula**

Esta célula é responsável por carregar e fazer uma análise inicial do seu dataset de emoções. Veja o que ela faz:
1. Carregamento do Dataset do GitHub: Ela primeiro imprime uma mensagem de "Carregando dataset do GitHub..." e então usa a biblioteca requests para baixar um arquivo JSON de uma URL específica no GitHub. Este arquivo contém as frases e suas segmentações emocionais.
2. Confirmação do Carregamento: Após o download, ela imprime uma mensagem confirmando que o dataset foi carregado com sucesso e quantos exemplos ele contém (no seu caso, 307 exemplos).
3. Exibição de Exemplos: Para ajudar a visualizar a estrutura dos dados, a célula mostra os três primeiros exemplos do dataset. Para cada exemplo, ela exibe a frase original e o segmented_text, que é a mesma frase com as tags de emoção (<EMOCAO-Inicio>, <EMOCAO-Fim>) inseridas.
4. Análise da Distribuição das Emoções: Este é um passo importante para entender a composição do seu dataset. Ela faz o seguinte:
    * Usa uma expressão regular (re.findall(r'<([A-Z_]+)-Inicio>', item['segmented_text'])) para encontrar todas as tags de início de emoção (como <FELIZ-Inicio>, <MEDO-Inicio>) em cada segmented_text.
    * A biblioteca collections.Counter é usada para contar a frequência de cada emoção encontrada. Atenção: Na última interação, corrigi a expressão regular para Inicio (sem acento) para corresponder ao formato real do dataset, o que resolveu o problema de saída incompleta que você mencionou.
    * Finalmente, calcula a porcentagem de cada emoção em relação ao total de segmentos emocionais e imprime um resumo da distribuição (ex.: FELIZ: 149 segmentos (23.8%), etc.).

Em resumo, esta célula prepara os dados, mostrando o que você está trabalhando e dando uma visão geral da frequência de cada emoção no seu conjunto de dados.

In [ ]:
#2. carregando o dataset do repositório do github
print("📥 Carregando dataset do GitHub...")
github_url = "https://raw.githubusercontent.com/liviafernanda/pln-inf-ufg/refs/heads/main/hannah_dataset_pt-br.json"

try:
    response = requests.get(github_url)
    data = response.json()
    print(f"✅ Dataset carregado: {len(data)} exemplos")

    for item in data[:3]:  # Mostrar primeiros 3 exemplos
        print(f"\n📝 Exemplo " + str(data.index(item) + 1) + ":")
        print(f"   Frase: {item['sentence']}")
        print(f"   Segmentado: {item['segmented_text']}")

    # Análise da distribuição
    print("\n📊 ANALISANDO DISTRIBUIÇÃO DAS EMOÇÕES:")
    emotion_counts = Counter()
    for item in data:
        emotions = re.findall(r'<([A-Z_]+)-Inicio>', item['segmented_text'])
        emotion_counts.update(emotions)

    total_segments = sum(emotion_counts.values())
    for emotion, count in emotion_counts.most_common():
        percentage = (count / total_segments) * 100
        print(f"   {emotion}: {count} segmentos ({percentage:.1f}%)")

except Exception as e:
    print(f"❌ Erro: {e}")

📥 Carregando dataset do GitHub...
✅ Dataset carregado: 307 exemplos

📝 Exemplo 1:
   Frase: Depois de semanas tentando resolver o problema, o código finalmente rodou sem erros, e por um instante tudo pareceu fazer sentido, mas então percebi que o resultado não correspondia ao esperado.
   Segmentado: <FELIZ-Inicio>Depois de semanas tentando resolver o problema, o código finalmente rodou sem erros, e por um instante tudo pareceu fazer sentido<FELIZ-Fim><TRISTE-Inicio>, mas então percebi que o resultado não correspondia ao esperado.<TRISTE-Fim>

📝 Exemplo 2:
   Frase: As crianças corriam pelo parque, rindo alto e espalhando folhas pelo chão, até que o trovão estrondou e todos correram para se abrigar.
   Segmentado: <FELIZ-Inicio>As crianças corriam pelo parque, rindo alto e espalhando folhas pelo chão<FELIZ-Fim><MEDO-Inicio>, até que o trovão estrondou e todos correram para se abrigar.<MEDO-Fim>

📝 Exemplo 3:
   Frase: Eu esperava encontrar a casa como a deixei, mas ao abrir a porta, tu

> **3ª célula**

Esta célula define a classe EmotionDataset, que é fundamental para preparar seus dados brutos de texto para o modelo BERT. Ela faz o seguinte:
1. Propósito da Classe (EmotionDataset): Esta classe herda de torch.utils.data.Dataset, o que significa que ela foi projetada para interagir com o DataLoader do PyTorch. O objetivo é transformar cada frase do seu dataset em um formato numérico que o modelo BERT possa entender, ou seja, input_ids (tokens numéricos), attention_mask (para indicar o que é texto real e o que é preenchimento) e labels (as emoções correspondentes a cada token).
2. Inicialização (__init__):
    * Recebe os dados brutos (data), o tokenizer do BERT e um max_length (tamanho máximo das sequências).
    * Define mapeamentos entre nomes de emoções e IDs numéricos (emotion_to_id) e vice-versa (id_to_emotion). Isso é crucial porque os modelos de Machine Learning trabalham com números, não com strings.
    * Imprime o mapeamento das emoções para que você possa ver qual número corresponde a cada emoção.
    * Chama o método _process_data() para pré-processar todo o dataset assim que a classe é instanciada.
3. Processamento dos Dados (_process_data):
    * Este método itera sobre cada item (frase) do seu dataset original.
    * Para cada frase, ele usa o tokenizer para convertê-la em input_ids (IDs dos tokens), attention_mask (máscara de atenção) e aplica padding (preenchimento) ou truncation (corte) para que todas as sequências tenham o max_length definido.
    * Em seguida, ele chama _extract_token_labels() para obter as etiquetas de emoção para cada token na frase.
    * Armazena os input_ids, attention_mask e labels de cada frase em uma lista processed.
4. Extração de Etiquetas de Tokens (_extract_token_labels):
    * Esta é a parte mais complexa: o objetivo é atribuir uma emoção a CADA token da frase.
    * Começa inicializando todas as etiquetas com 0 (que corresponde a NEUTRO).
    * Usa uma expressão regular (r'<([A-Z_]+)-Inicio>(.*?)< -Fim>') para encontrar os segmentos de texto marcados com emoções (ex: <FELIZ-Inicio>...<FELIZ-Fim>) no segmented_text.
    * Para cada segmento emocional encontrado:
        * Tokeniza o texto desse segmento.
        * Procura a sequência desses tokens dentro da lista completa de tokens da frase (obtida do tokenizer).
        * Uma vez encontrada a correspondência, ele atribui o emotion_id (número da emoção) correspondente a todos os tokens desse segmento.
    * Tratamento de Tokens Especiais: Tokens como [CLS] (início da sequência), [SEP] (separador de sequência) e [PAD] (preenchimento) não devem contribuir para o cálculo da perda do modelo. Por isso, suas etiquetas são definidas como -100, que é um valor especial que o CrossEntropyLoss (função de perda) ignora por padrão.
    5. Métodos __len__ e __getitem__:
    * __ len__: Retorna o número total de exemplos processados no dataset, permitindo que o DataLoader saiba quantos itens existem.
    * __ getitem__: Permite acessar um item específico do dataset pelo seu índice, o que é essencial para que o DataLoader possa carregar os dados em lotes (batches).
    
Em resumo, esta classe é a ponte entre seus dados de texto brutos e o formato numérico estruturado que o modelo BERT-LSTM espera para treinamento, garantindo que cada token tenha uma etiqueta de emoção correspondente.

In [ ]:
#3. pre processamento do dataseet
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.emotion_to_id = {
            'NEUTRO': 0, 'FELIZ': 1, 'TRISTE': 2,
            'RAIVA': 3, 'MEDO': 4, 'SURPRESA': 5
        }
        self.id_to_emotion = {v: k for k, v in self.emotion_to_id.items()}

        print("🎭 Mapeamento de emoções:")
        for emotion, eid in self.emotion_to_id.items():
            print(f"   {emotion} -> {eid}")

        self.processed_data = self._process_data()

    def _process_data(self):
        processed = []
        print("🔄 Processando dados...")

        for item in tqdm(self.data, desc="Processando"):
            sentence = item['sentence']
            segmented_text = item['segmented_text']

            encoding = self.tokenizer(
                sentence,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )

            input_ids = encoding['input_ids'][0]
            tokens = self.tokenizer.convert_ids_to_tokens(input_ids)
            token_labels = self._extract_token_labels(tokens, segmented_text)

            processed.append({
                'input_ids': encoding['input_ids'][0],
                'attention_mask': encoding['attention_mask'][0],
                'labels': torch.tensor(token_labels, dtype=torch.long)
            })

        return processed

    def _extract_token_labels(self, tokens, segmented_text):
        labels = [0] * len(tokens)  # NEUTRO como padrão

        try:
            pattern = r'<([A-Z_]+)-Inicio>(.*?)<\1-Fim>'
            matches = re.findall(pattern, segmented_text)

            for emotion, text in matches:
                text = text.strip()
                segment_tokens = self.tokenizer.tokenize(text)

                # Buscar o segmento na sentença
                for i in range(len(tokens) - len(segment_tokens) + 1):
                    if tokens[i:i+len(segment_tokens)] == segment_tokens:
                        emotion_id = self.emotion_to_id.get(emotion, 0)
                        for j in range(len(segment_tokens)):
                            if i + j < len(labels):
                                labels[i + j] = emotion_id
                        break

            # Marcar tokens especiais
            for i, token in enumerate(tokens):
                if token in [self.tokenizer.cls_token, self.tokenizer.sep_token, self.tokenizer.pad_token]:
                    labels[i] = -100

        except Exception as e:
            print(f"⚠️ Erro no processamento: {e}")

        return labels

    def __len__(self):
        return len(self.processed_data)

    def __getitem__(self, idx):
        return self.processed_data[idx]

> **4ª célula**

Esta célula define a arquitetura do seu modelo de Segmentação de Emoções, chamado EmotionSegmentationModel. Ele é uma combinação de um modelo BERT pré-treinado com uma camada LSTM e um classificador. Vamos detalhar cada parte:
1. Inicialização (__init__):
      * super(EmotionSegmentationModel, self).__ init__(): Chama o construtor da classe pai nn.Module.
      * print("🧠 Carregando BERT com fine-tuning..."): Indica o início do carregamento do modelo BERT.
      * self.bert = BertModel.from_pretrained(bert_model_name): Carrega um modelo BERT pré-treinado. No seu caso, ele usa o neuralmind/bert-base-portuguese-cased, que é um BERT base treinado para o português.
2. Fine-tuning Parcial do BERT:
      * for name, param in self.bert.named_parameters():: Itera sobre todos os parâmetros (pesos e vieses) do modelo BERT.
      * if any(layer in name for layer in ['layer.10', 'layer.11', 'pooler']):: Esta condição verifica se o parâmetro pertence às duas últimas camadas do BERT (layer.10, layer.11) ou à camada pooler. Estas são as camadas mais próximas da saída do BERT e, geralmente, as mais específicas para a tarefa de fine-tuning.
      * param.requires_grad = True: Para essas camadas específicas, requires_grad é definido como True. Isso significa que os pesos dessas camadas serão atualizados durante o treinamento (fine-tuning).
      * else: param.requires_grad = False: Para as demais camadas do BERT (as mais baixas), requires_grad é definido como False. Isso 'congela' os pesos dessas camadas, impedindo que sejam alterados durante o treinamento. Essa técnica ajuda a preservar o conhecimento geral do BERT e a acelerar o treinamento, focando as atualizações nas camadas mais relevantes para a sua tarefa específica.
3. Camada LSTM Bidirecional (self.lstm):
      * nn.LSTM(...): Após o BERT extrair características contextuais do texto, uma camada Long Short-Term Memory (LSTM) é adicionada. LSTMs são redes neurais recorrentes eficazes para processar sequências.
      * input_size=self.bert.config.hidden_size: A entrada para a LSTM é a saída de cada token do BERT (que tem hidden_size dimensões).
      * hidden_size=hidden_size: Define o tamanho da saída oculta de cada unidade LSTM.
      * num_layers=num_layers: Define quantas camadas LSTM são empilhadas.
      * bidirectional=True: Muito importante! Significa que a LSTM processa a sequência em duas direções (da esquerda para a direita e da direita para a esquerda). Isso permite que o modelo capture o contexto de um token a partir de palavras que vêm antes e depois dele, o que é crucial para entender nuances emocionais.
      * batch_first=True: Indica que a dimensão do batch (tamanho do lote de dados) é a primeira.
      * dropout: Ajuda a prevenir overfitting, zerando aleatoriamente algumas conexões durante o treinamento.
4. Classificador Melhorado (self.classifier):
      * nn.Sequential(...): Um módulo sequencial que organiza as camadas do classificador.
      * nn.Dropout(dropout): Uma camada de dropout inicial para regularização.
      * nn.Linear(hidden_size * 2, hidden_size): Uma camada linear (totalmente conectada). Como a LSTM é bidirecional, sua saída tem hidden_size * 2 dimensões, que são mapeadas para hidden_size.
      * nn.ReLU(): Uma função de ativação Rectified Linear Unit, que introduz não-linearidade no modelo.
      * nn.Dropout(dropout): Outra camada de dropout para mais regularização.
      * nn.Linear(hidden_size, num_emotions): A camada de saída final. Mapeia a saída da camada anterior para o número de classes de emoção (num_emotions) que você tem. A saída desta camada são os 'logits', que serão usados para calcular as probabilidades de cada emoção por token.
5. Passagem forward: Este método descreve como os dados fluem através do modelo durante a inferência ou treinamento:
      * bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask): Primeiro, os input_ids e attention_mask são passados para o modelo BERT. Ele retorna várias saídas, e last_hidden_state é a sequência de representações contextuais para cada token.
      * lstm_output, _ = self.lstm(sequence_output): A saída do BERT (sequence_output) é então alimentada à camada LSTM bidirecional.
      * logits = self.classifier(lstm_output): A saída da LSTM é passada para o classificador, que produz os logits para cada token, indicando a probabilidade de cada emoção.

Em resumo, este modelo aproveita o poderoso entendimento contextual de um BERT pré-treinado em português, refina esse entendimento com camadas LSTM bidirecionais para capturar dependências de longo alcance na sequência, e então usa um classificador para prever a emoção de cada token individualmente. A estratégia de fine-tuning parcial otimiza o uso dos recursos e o tempo de treinamento.

In [ ]:
#4. Estrutura do modelo
class EmotionSegmentationModel(nn.Module):
    def __init__(self, bert_model_name, num_emotions, hidden_size=256, num_layers=2, dropout=0.3):
        super(EmotionSegmentationModel, self).__init__()

        print("🧠 Carregando BERT com fine-tuning...")
        self.bert = BertModel.from_pretrained(bert_model_name)

        # FINE-TUNING PARCIAL: descongelar últimas camadas
        for name, param in self.bert.named_parameters():
            if any(layer in name for layer in ['layer.10', 'layer.11', 'pooler']):
                param.requires_grad = True
            else:
                param.requires_grad = False

        self.lstm = nn.LSTM(
            input_size=self.bert.config.hidden_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # CLASSIFICADOR MELHORADO
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_emotions)
        )

    def forward(self, input_ids, attention_mask):
        # COM fine-tuning nas últimas camadas
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs.last_hidden_state

        lstm_output, _ = self.lstm(sequence_output)
        logits = self.classifier(lstm_output)

        return logits

> **5ª célula**

Esta célula contém as funções centrais para o treinamento, avaliação e uso do seu modelo de segmentação de emoções. Vamos detalhar cada uma:
1. train_model_improved(model, train_loader, val_loader, num_epochs=10, learning_rate=0.0005, patience=3)
Esta é a função principal de treinamento do modelo. Ela otimiza o modelo para aprender a mapear texto para emoções. Veja o que ela faz:
    * Otimizador (optimizer): Usa AdamW, um otimizador popular para modelos baseados em Transformers, para ajustar os pesos do modelo com base na perda.
    * Função de Perda (criterion): Emprega CrossEntropyLoss, que é adequada para problemas de classificação multi-classe. O ignore_index=-100 é importante, pois indica que tokens com label -100 (os tokens especiais como [CLS], [SEP], [PAD] que você configurou na classe EmotionDataset) não contribuem para o cálculo da perda.
    * Loop de Treinamento: Itera sobre o número de num_epochs (épocas) definidos.
        * Mini-batches: Dentro de cada época, o modelo processa os dados em mini-batches do train_loader.
        * Forward Pass: Os input_ids e attention_mask são passados para o modelo (logits = model(input_ids, attention_mask)), que retorna as previsões (logits).
        * Cálculo da Perda: A perda é calculada comparando os logits com os labels reais.
        * Backward Pass e Otimização: A perda é retropropagada (loss.backward()) para calcular os gradientes, e o otimizador (optimizer.step()) atualiza os pesos do modelo.
        * Clipping de Gradiente: torch.nn.utils.clip_grad_norm_ é usado para evitar 'gradientes explosivos', um problema comum em redes neurais profundas.
    * Validação e Early Stopping: Após cada época de treinamento, o modelo é avaliado no conjunto de validação usando evaluate_model_colab. Se a acurácia de validação melhorar, o modelo atual é salvo como o 'melhor modelo'. Se a acurácia não melhorar por um número de épocas (patience), o treinamento é interrompido (early stopping) para evitar overfitting.
    * Cachê da GPU: torch.cuda.empty_cache() libera a memória da GPU após cada época, ajudando a evitar problemas de memória.
    * Carregar Melhor Modelo: Ao final do treinamento, o modelo com a melhor performance de validação é carregado novamente, garantindo que você tenha a melhor versão treinada.
2. evaluate_model_colab(model, data_loader)
Esta função é usada para avaliar o desempenho do modelo em um conjunto de dados (geralmente o de validação ou teste) e calcular a acurácia. Ela faz o seguinte:
    * Modo de Avaliação: Define o modelo em model.eval() para desativar recursos como Dropout, que só são usados durante o treinamento.
    * Inferência sem Gradientes: Usa torch.no_grad() para desativar o cálculo de gradientes, o que economiza memória e acelera a inferência.
    * Previsões: Para cada batch no data_loader, o modelo faz previsões e calcula as classes mais prováveis (torch.argmax(logits, dim=-1)).
    * Filtragem de Labels Ignorados: As previsões e labels são filtradas para remover os tokens com label -100 (tokens especiais).
    * Cálculo da Acurácia: accuracy_score da scikit-learn é usado para calcular a acurácia geral do modelo nas previsões válidas.
    * Voltar ao Modo de Treinamento: Retorna o modelo para model.train() ao final da avaliação.
3. predict_emotion_segments_improved(model, tokenizer, sentence, id_to_emotion, max_length=128, min_confidence=0.4)
Esta é a função que você usará para fazer previsões com o modelo treinado em uma nova frase. Ela não apenas prevê, mas também pós-processa os resultados de forma inteligente:
    * Modo de Avaliação: Coloca o modelo em model.eval().
    * Tokenização e Previsão: A frase de entrada é tokenizada pelo tokenizer e passada para o modelo para obter os logits e as probabilities (probabilidades de cada emoção para cada token).
    * Extração de Tokens e Confiança: Converte os IDs dos tokens de volta para strings e extrai as previsões de emoção e as probabilidades máximas para cada token.
    * Pós-processamento Inteligente: Este é o coração desta função, projetado para criar segmentos de emoção mais coesos e compreensíveis:
        * Agrupamento de Emoções: Itera sobre os tokens e agrupa tokens consecutivos com a mesma emoção para formar segmentos.
        * Confiança Mínima (min_confidence): Se a confiança de uma predição for abaixo de min_confidence, o modelo tenta manter a emoção do token anterior, o que ajuda a suavizar transições e a ignorar 'ruído' de baixa confiança.
        * Filtragem de Segmentos Pequenos: Descarta segmentos de texto muito curtos que não contêm pelo menos 3 caracteres úteis.
        * Mesclagem de Segmentos Pequenos Consecutivos: Se dois segmentos consecutivos têm a mesma emoção e são ambos muito curtos (menos de 2 palavras), eles são mesclados em um único segmento. Isso ajuda a evitar segmentações excessivamente fragmentadas.
    * Retorno dos Segmentos: Retorna uma lista de dicionários, onde cada dicionário representa um segmento de emoção detectado, contendo o text (texto do segmento), emotion (emoção atribuída) e confidence (confiança média da emoção para aquele segmento).
    
Em resumo, essas funções formam o pipeline completo para treinar um modelo de segmentação de emoções, monitorar seu desempenho e, em seguida, usá-lo para analisar novas frases, incluindo um pós-processamento inteligente para gerar resultados mais coerentes.

In [ ]:
#5. Funções de treino
def train_model_improved(model, train_loader, val_loader, num_epochs=10, learning_rate=0.0005, patience=3):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    criterion = nn.CrossEntropyLoss(ignore_index=-100)

    best_accuracy = 0
    patience_counter = 0
    train_losses = []
    val_accuracies = []

    model.train()

    print(f"🚀 Iniciando treinamento com {num_epochs} épocas...")

    for epoch in range(num_epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f'Época {epoch+1}/{num_epochs}')

        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            logits = model(input_ids, attention_mask)
            loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'perda': f'{loss.item():.4f}'})

        epoch_loss = total_loss / len(train_loader)
        train_losses.append(epoch_loss)

        # VALIDAÇÃO
        val_accuracy = evaluate_model_colab(model, val_loader)
        val_accuracies.append(val_accuracy)

        print(f'📊 Época {epoch+1} - Perda: {epoch_loss:.4f}, Acurácia Val: {val_accuracy:.4f}')

        # EARLY STOPPING
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            patience_counter = 0
            torch.save(model.state_dict(), 'melhor_modelo.pt')
            print(f'💾 Melhor modelo salvo (acurácia: {val_accuracy:.4f})')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'🛑 Early stopping na época {epoch+1}')
                break

        torch.cuda.empty_cache()

    # Carregar melhor modelo
    model.load_state_dict(torch.load('melhor_modelo.pt'))
    print(f'✅ Melhor modelo carregado (acurácia: {best_accuracy:.4f})')

    return train_losses, val_accuracies

def evaluate_model_colab(model, data_loader):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Validando"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            predictions = torch.argmax(logits, dim=-1)

            mask = labels != -100
            valid_predictions = predictions[mask].cpu().numpy()
            valid_labels = labels[mask].cpu().numpy()

            all_predictions.extend(valid_predictions)
            all_labels.extend(valid_labels)

    accuracy = accuracy_score(all_labels, all_predictions) if all_labels else 0.0
    model.train()

    return accuracy

def predict_emotion_segments_improved(model, tokenizer, sentence, id_to_emotion, max_length=128, min_confidence=0.4):
    """Versão melhorada com pós-processamento inteligente"""
    model.eval()

    encoding = tokenizer(
        sentence,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=-1)
        max_probs, predictions = torch.max(probabilities, dim=-1)

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    predictions = predictions[0].cpu().numpy()
    max_probs = max_probs[0].cpu().numpy()

    # PÓS-PROCESSAMENTO INTELIGENTE
    segments = []
    current_emotion = None
    current_tokens = []
    current_confidences = []

    for i, (token, emotion_id, confidence) in enumerate(zip(tokens, predictions, max_probs)):
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue

        emotion = id_to_emotion[emotion_id]

        # SE CONFIANÇA BAIXA, MANTER EMOÇÃO ANTERIOR
        if confidence < min_confidence and current_emotion:
            emotion = current_emotion

        if emotion != current_emotion:
            if current_tokens:
                segment_text = tokenizer.convert_tokens_to_string(current_tokens).strip()
                avg_confidence = np.mean(current_confidences) if current_confidences else 0.0

                # FILTRAR SEGMENTOS MUITO PEQUENOS (menos de 2 caracteres úteis)
                if len(segment_text) >= 3:
                    segments.append({
                        'text': segment_text,
                        'emotion': current_emotion,
                        'confidence': avg_confidence
                    })

            current_emotion = emotion
            current_tokens = [token]
            current_confidences = [confidence]
        else:
            current_tokens.append(token)
            current_confidences.append(confidence)

    # ÚLTIMO SEGMENTO
    if current_tokens:
        segment_text = tokenizer.convert_tokens_to_string(current_tokens).strip()
        avg_confidence = np.mean(current_confidences) if current_confidences else 0.0
        if len(segment_text) >= 3:
            segments.append({
                'text': segment_text,
                'emotion': current_emotion,
                'confidence': avg_confidence
            })

    # MESCLAR SEGMENTOS PEQUENOS CONSECUTIVOS DA MESMA EMOÇÃO
    if len(segments) > 1:
        merged_segments = []
        i = 0
        while i < len(segments):
            current = segments[i]
            if i < len(segments) - 1:
                next_seg = segments[i + 1]
                # Se segmentos consecutivos da mesma emoção e ambos pequenos, mesclar
                if (current['emotion'] == next_seg['emotion'] and
                    len(current['text'].split()) <= 2 and
                    len(next_seg['text'].split()) <= 2):
                    merged_text = current['text'] + ' ' + next_seg['text']
                    merged_confidence = (current['confidence'] + next_seg['confidence']) / 2
                    merged_segments.append({
                        'text': merged_text.strip(),
                        'emotion': current['emotion'],
                        'confidence': merged_confidence
                    })
                    i += 2  # Pular o próximo
                else:
                    merged_segments.append(current)
                    i += 1
            else:
                merged_segments.append(current)
                i += 1
        segments = merged_segments

    return segments

> **6ª célula**

Esta célula é o coração do seu processo de treinamento, onde tudo o que foi definido nas células anteriores é juntado para treinar o modelo de segmentação de emoções. Vamos ver o que acontece passo a passo:
  1. Início do Treinamento: A célula começa com uma mensagem clara: "=== 🚀 INICIANDO TREINAMENTO OTIMIZADO ===" para indicar o começo do processo.
  2. Configuração do BERT (bert_model_name): Define o nome do modelo BERT que será utilizado. No seu caso, é o neuralmind/bert-base-portuguese-cased, que é uma versão do BERT treinada especificamente para o português.
  3. Carregamento do Tokenizer: O BertTokenizer.from_pretrained(bert_model_name) carrega o tokenizador correspondente ao modelo BERT escolhido. O tokenizador é essencial para converter seu texto em sequências numéricas (tokens e IDs) que o BERT pode processar. Uma mensagem "✅ Tokenizer carregado" confirma o sucesso.
  4. Criação do Dataset: Uma instância da classe EmotionDataset (que definimos na célula 3) é criada, passando os dados brutos (data carregado da célula 2) e o tokenizer. Esta etapa pré-processa todo o seu dataset, convertendo as frases e suas anotações de emoção em input_ids, attention_mask e labels prontos para o treinamento. Uma mensagem "✅ Dataset criado" indica que o processamento inicial foi concluído.
  5. Divisão Treino/Validação (train_test_split):
      * O dataset total é dividido em dois subconjuntos: 80% para treinamento (train_dataset) e 20% para validação (val_dataset).
      * torch.utils.data.random_split garante que essa divisão seja feita de forma aleatória, o que é importante para que ambos os conjuntos sejam representativos.
      * As variáveis train_loader e val_loader são criadas usando DataLoader. Elas são responsáveis por carregar os dados em 'lotes' (batches) para o treinamento e validação, respectivamente, e podem embaralhar os dados (shuffle=True para treino) para melhorar o aprendizado do modelo.
      * Uma mensagem "📊 Divisão: ... treino, ... validação" mostra o tamanho de cada conjunto.
  6. Inicialização do Modelo (EmotionSegmentationModel):
      * Uma instância da classe EmotionSegmentationModel (definida na célula 4) é criada. São passados parâmetros como o nome do modelo BERT, o número de emoções (obtido do dataset.emotion_to_id), o tamanho da camada oculta da LSTM (hidden_size), o número de camadas LSTM e o dropout.
      * .to(device) move o modelo para o dispositivo de computação (CPU ou GPU) que foi detectado na primeira célula, garantindo que as operações sejam executadas no hardware correto.
      * A célula imprime "🧠 Modelo criado" e "📐 Parâmetros treináveis:", mostrando quantos parâmetros do modelo serão ajustados durante o treinamento (lembre-se que algumas camadas do BERT foram congeladas).
  7. Treinamento Otimizado (train_model_improved):
      * A função train_model_improved (definida na célula 5) é chamada para iniciar o processo de treinamento real. Ela recebe o modelo, os data loaders de treino e validação, o número de épocas, a taxa de aprendizado e o parâmetro de patience para o early stopping.
      * Esta função gerencia todo o ciclo de treinamento: forward pass, cálculo de perda, backward pass, atualização de pesos, avaliação em validação e salvamento do melhor modelo.
  8. Conclusão: Após o término da função train_model_improved (seja por atingir o número máximo de épocas ou por early stopping), a célula imprime "💾 Treinamento concluído!" indicando que o processo principal foi finalizado.

Em resumo, esta célula coordena todas as etapas para construir, treinar e otimizar seu modelo de segmentação de emoções, usando as configurações e funções que você definiu nas células anteriores.

In [ ]:
#6. Treinamento principal
print("=== 🚀 INICIANDO TREINAMENTO OTIMIZADO ===")

# CONFIGURAÇÕES MELHORADAS
bert_model_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
print("✅ Tokenizer carregado")

dataset = EmotionDataset(data, tokenizer)
print("✅ Dataset criado")

# SPLIT TREINO/VALIDAÇÃO
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

print(f"📊 Divisão: {train_size} treino, {val_size} validação")

# MODELO MELHORADO
model = EmotionSegmentationModel(
    bert_model_name=bert_model_name,
    num_emotions=len(dataset.emotion_to_id),
    hidden_size=256,
    num_layers=2,
    dropout=0.3
).to(device)

print(f"🧠 Modelo criado")
print(f"📐 Parâmetros treináveis: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# TREINAMENTO OTIMIZADO
train_losses, val_accuracies = train_model_improved(
    model,
    train_loader,
    val_loader,
    num_epochs=10,
    learning_rate=0.0005,
    patience=4
)

print("💾 Treinamento concluído!")

=== 🚀 INICIANDO TREINAMENTO OTIMIZADO ===
✅ Tokenizer carregado
🎭 Mapeamento de emoções:
   NEUTRO -> 0
   FELIZ -> 1
   TRISTE -> 2
   RAIVA -> 3
   MEDO -> 4
   SURPRESA -> 5
🔄 Processando dados...


Processando: 100%|██████████| 307/307 [00:00<00:00, 446.93it/s]


✅ Dataset criado
📊 Divisão: 245 treino, 62 validação
🧠 Carregando BERT com fine-tuning...
🧠 Modelo criado
📐 Parâmetros treináveis: 18,577,414
🚀 Iniciando treinamento com 10 épocas...


Validando: 100%|██████████| 8/8 [00:23<00:00,  2.92s/it]


📊 Época 1 - Perda: 1.6604, Acurácia Val: 0.4632
💾 Melhor modelo salvo (acurácia: 0.4632)


...

📊 Época 8 - Perda: 0.4793, Acurácia Val: 0.6685


Validando: 100%|██████████| 8/8 [00:23<00:00,  2.89s/it]


📊 Época 9 - Perda: 0.4257, Acurácia Val: 0.7117
💾 Melhor modelo salvo (acurácia: 0.7117)


Validando: 100%|██████████| 8/8 [00:23<00:00,  2.89s/it]


📊 Época 10 - Perda: 0.3063, Acurácia Val: 0.7117
✅ Melhor modelo carregado (acurácia: 0.7117)
💾 Treinamento concluído!


> **7ª célula**

Teste de demonstração automático: possui algumas sentenças de teste para ser executado manualmente.

In [ ]:
#7. Teste de demonstração
def demonstrar_modelo_melhorado():
    print("\n" + "="*60)
    print("🎭 ANALISADOR DE EMOÇÕES - VERSÃO MELHORADA")
    print("="*60)
    print("✅ Modelo otimizado carregado!")
    print("💡 Digite frases em português para análise")
    print("🚪 Digite 'sair' para encerrar")
    print("-"*60)

    while True:
        frase = input("\n📝 Digite uma frase: ").strip()

        if frase.lower() in ['sair', 'exit', 'quit']:
            print("👋 Até logo!")
            break

        if not frase:
            continue

        print("⏳ Analisando com pós-processamento inteligente...")
        segmentos = predict_emotion_segments_improved(
            model, tokenizer, frase, dataset.id_to_emotion,
            min_confidence=0.4
        )

        print(f"\n✅ RESULTADO: '{frase}'")
        if not segmentos:
            print("   🤔 Nenhum segmento emocional detectado com confiança suficiente")
        else:
            print("🎭 Segmentos emocionais detectados:")
            for i, seg in enumerate(segmentos, 1):
                emoji = {
                    'FELIZ': '😊', 'TRISTE': '😢', 'RAIVA': '😠',
                    'MEDO': '😨', 'SURPRESA': '😲', 'NEUTRO': '😐'
                }.get(seg['emotion'], '❓')
                conf_str = f"(conf: {seg['confidence']:.2f})" if 'confidence' in seg else ""
                print(f"   {i}. {emoji} [{seg['emotion']}] {conf_str} → '{seg['text']}'")

        print("-" * 60)

# TESTAR COM EXEMPLOS PRÉ-DEFINIDOS PRIMEIRO
print("\n🧪 TESTANDO COM EXEMPLOS:")
exemplos_teste = [
    "Estou muito feliz com a notícia, mas depois fiquei triste ao pensar nas consequências.",
    "O filme foi assustador no início, mas terminou de forma surpreendente e feliz.",
    "Estou com raiva dessa situação, mas vou tentar manter a calma.",
    "Que susto! Tive um grande medo quando ouvi aquele barulho.",
    "Comecei uma caminhada na praia chorando pelo ocorrido, porém me alegrou ver as pessoas jogando bola e passeando com os cachorros.",
    "Uma mulher muito bonita me abordou na entrada solicitando meus documentos, e fiquei indignada por não terem me avisado dessa exigência antes."
]

for frase in exemplos_teste:
    print(f"\n📝 Frase: {frase}")
    segmentos = predict_emotion_segments_improved(model, tokenizer, frase, dataset.id_to_emotion)
    for i, seg in enumerate(segmentos, 1):
        emoji = {'FELIZ': '😊', 'TRISTE': '😢', 'RAIVA': '😠',
                'MEDO': '😨', 'SURPRESA': '😲', 'NEUTRO': '😐'}.get(seg['emotion'], '❓')
        print(f"   {i}. {emoji} [{seg['emotion']}] → '{seg['text']}'")

# INICIAR MODO INTERATIVO
#demonstrar_modelo_melhorado()


🧪 TESTANDO COM EXEMPLOS:

📝 Frase: Estou muito feliz com a notícia, mas depois fiquei triste ao pensar nas consequências.
   1. 😊 [FELIZ] → 'Estou muito feliz com a notícia'
   2. 😢 [TRISTE] → ', mas depois fiquei triste ao pensar nas consequências .'

📝 Frase: O filme foi assustador no início, mas terminou de forma surpreendente e feliz.
   1. 😊 [FELIZ] → 'O filme'
   2. 😲 [SURPRESA] → 'foi assustador'
   3. 😊 [FELIZ] → 'no início , mas terminou de forma surpreendente e feliz .'

📝 Frase: Estou com raiva dessa situação, mas vou tentar manter a calma.
   1. 😠 [RAIVA] → 'Estou com raiva dessa situação'
   2. 😐 [NEUTRO] → ', mas vou tentar manter a calma .'

📝 Frase: Que susto! Tive um grande medo quando ouvi aquele barulho.
   1. 😨 [MEDO] → 'Que susto ! Tive um grande medo quando ouvi aquele barulho .'

📝 Frase: Comecei uma caminhada na praia chorando pelo ocorrido, porém me alegrou ver as pessoas jogando bola e passeando com os cachorros.
   1. 😠 [RAIVA] → 'Comecei'
   2. 😊 [FELIZ] → 

> **8ª célula (final)**

Função simples de teste interativo com o usuário, para demonstração em tempo real. Solicita que o usuário insira uma sentença e faz a análise de sentimento.

In [ ]:
# CÉLULA FINAL - FUNÇÃO SIMPLES PARA TESTAR O MODELO

def analisar_sentimento():
    """
    Função simples que recebe uma frase do usuário e mostra a análise de sentimentos
    """
    print("\n" + "="*50)
    print("🎭 ANALISADOR DE SENTIMENTOS")
    print("="*50)

    # Carregar modelo treinado (assumindo que já foi treinado)
    try:
        # Se você já treinou o modelo nesta sessão, use estas variáveis:
        # model, tokenizer, dataset já devem estar definidos

        print("✅ Modelo carregado! Digite suas frases para análise.")
        print("💡 Digite 'sair' para encerrar")
        print("-"*50)

        while True:
            # Receber frase do usuário
            frase = input("\n📝 Digite uma frase: ").strip()

            if frase.lower() in ['sair', 'exit', 'quit']:
                print("👋 Até logo!")
                break

            if not frase:
                print("⚠️ Por favor, digite uma frase.")
                continue

            # Fazer a análise
            print("⏳ Analisando...")
            segmentos = predict_emotion_segments_improved(
            model, tokenizer, frase, dataset.id_to_emotion,
            min_confidence=0.4)

            # Mostrar resultado
            print(f"\n✅ RESULTADO PARA: '{frase}'")
            print("🎭 Segmentos emocionais detectados:")

            for i, segmento in enumerate(segmentos, 1):
                emoji = {
                    'FELIZ': '😊',
                    'TRISTE': '😢',
                    'COM_RAIVA': '😠',
                    'MEDO': '😨',
                    'SURPRESA': '😲',
                    'NEUTRO': '😐'
                }.get(segmento['emotion'], '❓')

                print(f"   {i}. {emoji} [{segmento['emotion']}] → '{segmento['text']}'")

            print("-" * 50)

    except NameError:
        print("❌ Modelo não encontrado. Execute o treinamento primeiro!")
    except Exception as e:
        print(f"❌ Erro: {e}")

analisar_sentimento()

# PARA USAR:
# 1. Primeiro execute todo o código de treinamento
# 2. Depois execute esta célula
# 3. Chame a função: analisar_sentimento()


🎭 ANALISADOR DE SENTIMENTOS
✅ Modelo carregado! Digite suas frases para análise.
💡 Digite 'sair' para encerrar
--------------------------------------------------

📝 Digite uma frase: As crianças corriam pelo parque, rindo alto e espalhando folhas pelo chão, até que o trovão estrondou e todos correram para se abrigar
⏳ Analisando...

✅ RESULTADO PARA: 'As crianças corriam pelo parque, rindo alto e espalhando folhas pelo chão, até que o trovão estrondou e todos correram para se abrigar'
🎭 Segmentos emocionais detectados:
   1. 😊 [FELIZ] → 'As crianças corriam pelo parque , rindo alto e espalhando folhas pelo chão'
   2. 😨 [MEDO] → ', até que o trovão estrondou e todos correram para se abrigar'
--------------------------------------------------

📝 Digite uma frase: sair
👋 Até logo!
